
# Gas Price Forecast


## 0.0. Requirements


### 0.1. Imports

In [0]:
import pandas as pd

from pytz import timezone
from datetime import datetime

from pyspark.sql import functions as pf


### 0.2. S3 Connection

In [0]:
# secrets is not available on Community =D
#access_key = dbutils.secrets.get(scope = "aws", key = "aws-access-key")
#secret_key = dbutils.secrets.get(scope = "aws", key = "aws-secret-key")
#bronze_bucket_name = dbutils.secrets.get(scope = "aws", key = "aws-bucket-bronze")
#silver_bucket_name = dbutils.secrets.get(scope = "aws", key = "aws-bucket-silver")

In [0]:
# encoded_secret_key = secret_key.replace("/", "%2F")
# aws_bucket_name = "example-aws-bucket"
# mount_name = "mount-name"
# 
# dbutils.fs.mount(
#     f"s3a://{access_key}:{encoded_secret_key}@{aws_bucket_name}", 
#     f"/mnt/{mount_name}"
# )

In [0]:
#spark.sql("CREATE EXTERNAL LOCATION [IF NOT EXISTS] <example-location-name-aws> URL 's3://<bucket-name>/<example-path>'
#     WITH (CREDENTIAL <aws-credential-name>)")

In [0]:
# Another Way
# sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", access_key)
# sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", secret_key)
# sc._jsc.hadoopConfiguration().set("fs.s3a.attempts.maximum", "3")
# sc._jsc.hadoopConfiguration().set("fs.s3a.connection.timeout", "10000")
# sc._jsc.hadoopConfiguration().set("fs.s3a.disable.chunked.encoding", "True")
# sc._jsc.hadoopConfiguration().set("fs.s3a.connection.establish.timeout", "5000")
# sc._jsc.hadoopConfiguration().set("spark.hadoop.fs.s3a.path.style.access", "True")
# sc._jsc.hadoopConfiguration().set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")


### 0.3. Data Load

In [0]:
date_file = datetime.now(tz=timezone("America/Sao_Paulo")).strftime("%Y%m%d")

df = spark.read.format("delta").load(f"s3://{bronze_bucket_name}/gas")
df = df.filter(pf.col("data_base") == date_file)

if not df.head(1):
    raise ValueError("Empty DataFrame")


## 1.0. Data Enrichment


### 1.1. Change Data Types

In [0]:
pop_cols = ["Date", "data_base"]
states = [k for k in df.columns if k not in ["Date", "data_base"]]

In [0]:
df = df.withColumn(
    "Date",
    pf.to_date(pf.col("Date"), "yyyy-MM-dd")
)

df = df.select(pop_cols + [pf.col(k).cast("double").alias(k) for k in states])


## 2.0. Save to Silver

In [0]:
df.write.format("delta").mode("overwrite").save(f"s3://{silver_bucket_name}/gas")